In [ ]:
import torch
import torch.nn as nn
import torch.jit

In [ ]:
def val_size(val):
    return val.type().sizes()

def val_inputs(val):
    return list(val.node().inputs())

def val_outputs(val):
    return list(val.node().outputs())

def val_kind(val):
    return val.node().kind()

def val_is_tensor(val):
    return val.isCompleteTensor()

In [ ]:
class MyModule(nn.Module):
    
    def __init__(self):
        super(MyModule, self).__init__()
        self.a = torch.ones(1)
        
    def forward(self, x):
        return x * torch.ones(1) * self.a